In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import torch
import torch.nn.functional as F
from torch import nn

import safetensors
from accelerate import notebook_launcher

from utils.data_utils import BrainDataset, get_tokenizer
from utils.train_utils import TrainConfig, run_train_model, count_parameters, load_model_weights

from dataclasses import dataclass
from simple_parsing.helpers import Serializable

from safetensors.torch import load_model
import albumentations as A

import matplotlib.pyplot as plt

In [2]:
from models.brain_mae import EncoderConfig, Encoder, MAE, MAEConfig

## Init brain module

In [3]:
encoder_config = EncoderConfig()
mae_config = MAEConfig()
model = MAE(encoder_config=encoder_config, mae_config=mae_config)


EncoderConfig(window_size=32, n_electrodes=256, patch_size=4, n_layers=12, dim=512, hidden_dim=1024, head_dim=32, n_heads=16, n_kv_heads=16)
Simple Encoder: number of parameters: 31.62M
MAEConfig(masking_ratio=0.75, n_layers=6, dim=256, hidden_dim=1024, head_dim=16, n_heads=8, n_kv_heads=8)
MAE: number of parameters: 37.79M


## Init language model

In [4]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
window_size = encoder_config.window_size
n_electrodes = 256

train_transform = A.Compose([
    
    # A.CoarseDropout(fill_value=0, p=0.5),
    # A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
    # A.GaussNoise(var_limit=0.005, mean=0, p=0.5),

    # A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
    #               border_mode=0, value=0, always_apply=True),
    A.RandomCrop(height=window_size, width=n_electrodes, always_apply=True),
    # A.Crop(x_min=0, x_max=n_electrodes, y_min=0, y_max=window_size, always_apply=True),

])

test_transform = A.Compose([
    A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
                  border_mode=0, value=0, always_apply=True),
    A.Crop(x_min=0, x_max=n_electrodes, y_min=0, y_max=window_size, always_apply=True)
])


data_path = Path("/drive/data/competitionData")

train_dataset = BrainDataset(data_path / 'train', tokenize_function=get_tokenizer(tokenizer), transform=train_transform)
test_dataset = BrainDataset(data_path / 'test', tokenize_function=get_tokenizer(tokenizer), transform=test_transform)

Runed processing of the  /drive/data/competitionData/train
len of the dataset: 8800
max signal size: 906 | max tokens size: 24
median signal size: 297.0 | median tokens size: 10.0
Runed processing of the  /drive/data/competitionData/test
len of the dataset: 880
max signal size: 919 | max tokens size: 21
median signal size: 283.5 | median tokens size: 9.0


In [6]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

x, y, date = next(iter(test_dataloader))
print(x.shape, y.shape, date.shape)

loss, logits = model(x, y, date)


torch.Size([1, 32, 256]) torch.Size([1, 25]) torch.Size([1, 1])


# Work with data

In [7]:
project_name = 'simple_mae'
save_folder = Path("/drive/logs/kovalev")

train_config = TrainConfig(exp_name='brain-mae-test-one',
                           mixed_precision=True,
                           batch_size=256, 
                           grad_accum=8,
                           num_workers=3,
                           pin_memory=True, 
                           eval_interval=250, 
                           learning_rate=1e-3,
                           weight_decay=0, 
                           grad_clip=10,
                           lr_decay_iters=20_000, 
                           warmup_iters=1000, 
                           project_name=project_name, 
                           save_folder=save_folder
                          )


args = (model, (train_dataset, test_dataset), train_config, encoder_config)
notebook_launcher(run_train_model, args, num_processes=1)

Launching training on one GPU.


/opt/conda/envs/pytorch/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


Device for training:  cuda
Num devices:  1
Completed initialization of scheduler
*********************************************************************************************************************************************************************************************************************************************************
overall_steps 250: 0.018509507179260254
val loss: 0.01854286901652813
saved model:  step_250_loss_0.0185.safetensors
**********************************************************************************************************************************************************************************************************************************************************
overall_steps 500: 0.01824205368757248
val loss: 0.018321990966796875
saved model:  step_500_loss_0.0183.safetensors
*****************

KeyboardInterrupt: 

In [15]:
all_features = []
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

for x, y, date in test_dataloader:
    x = x.to('cpiu')
    date = date.to('cuda')
    features = model.encoder(x, date_info=date)
    all_features.extend(features)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 21.96 GiB of which 1.12 MiB is free. Including non-PyTorch memory, this process has 21.96 GiB memory in use. Of the allocated memory 19.23 GiB is allocated by PyTorch, and 2.49 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

torch.Size([1, 512])